In [5]:
# Importamos las librerías necesarias
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats 
import seaborn as sns 
import warnings

from sklearn.model_selection import train_test_split 
#from sklearn.neighbors import KNeighborsRegressor
#from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensemble import ExtraTreesRegressor
#from xgboost import XGBRegressor
import xgboost as xgb
from IPython.display import display_html
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from imblearn.ensemble import BalancedBaggingClassifier

In [8]:
# Leemos el dataset original
original_df = pd.read_csv('../data/train.csv')

In [9]:
def transform_data(train_data_fname, test_data_fname):
    
    df_train = pd.read_csv(train_data_fname)
    df_train['is_train_set'] = 1
    
    df_test = pd.read_csv(test_data_fname)
    df_test['is_train_set'] = 0
    
    df_train.fillna({'Upc': original_df.Upc.mode()[0]}, inplace=True)
    df_test.fillna({'Upc': original_df.Upc.mode()[0]}, inplace=True)
    df_train['ItemCode'] = df_train['Upc'].apply(
      lambda x: str(x)[6:10])
    df_test['ItemCode'] = df_test['Upc'].apply(
      lambda x: str(x)[6:10]
    )
    # we  get the TripType for the train set. To do that, we group by VisitNumber and
    # then we get the max (or min or avg)
    y = df_train.groupby(["VisitNumber", "Weekday", "ItemCode"], as_index=False).max().TripType
    # we remove the TripType now, and concat training and testing data
    # the concat is done so that we have the same columns for both datasets
    # after one-hot encoding
    df_train = df_train.drop("TripType", axis=1)
    df = pd.concat([df_train, df_test])
    
    # the next three operations are the ones we have just presented in the previous lines
    
    # drop the columns we won't use (it may be good to use them somehow)
    df_notna = df.drop(["FinelineNumber"], axis=1)
    
    # one-hot encoding for the DepartmentDescription
    df_notna = pd.get_dummies(df_notna, columns=["DepartmentDescription"], dummy_na=True)
    # now we add the groupby values
    df_notna = df_notna.groupby(["VisitNumber", "Weekday", "ItemCode"], as_index=False).sum()
    
    # finally, we do one-hot encoding for the Weekday
    df_notna = pd.get_dummies(df_notna, columns=["Weekday"], dummy_na=True)
    # one-hot encoding for the ItemCode
    df_notna = pd.get_dummies(df_notna, columns=["ItemCode"], dummy_na=True)
    # get train and test back
    df_train = df_notna[df_notna.is_train_set != 0]
    df_test = df_notna[df_notna.is_train_set == 0]
    
    X = df_train.drop(["is_train_set"], axis=1)
    yy = None
    XX = df_test.drop(["is_train_set"], axis=1)
    return X, y, XX, yy

In [10]:
# Cargamos la información del dataset
X, y, XX, yy = transform_data("../data/train.csv", "../data/test.csv")

In [11]:
X.shape, y.shape, XX.shape

((423935, 10420), (423935,), (180832, 10420))